# PCP case study
## No chl prime

### Notebook helper function for pretty output

In [ ]:
from IPython.display import Markdown
from IPython.display import display


def print_md(markdown_printable):
    """Pretty render Markdown."""
    display(Markdown(str(markdown_printable)))


def print_yaml_file(file_path):
    """Pretty render yaml file."""
    with open(file_path) as f:
        print_md(f"```yaml\n{f.read()}\n```")

### Plotting functions ([`pyglotaran_extras`](https://github.com/s-weigand/pyglotaran-extras/commit/20da3593105fb839f86e668dc12dc9ca87c3b9ce) + `matplotlib`)

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (10, 7)

### Analysis functions

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset
from glotaran.io import load_model
from glotaran.io import load_parameters
from glotaran.project.scheme import Scheme

### Read data

In [ ]:
dataset = load_dataset("PCP490.ascii")
dataset

## Investigating the original data

In [ ]:
dataset.data.plot(x="time", y="spectral")

In [ ]:
last_traces = dataset.data.isel(
    time=slice(-3, None),
)
last_traces.plot.line(x="spectral", aspect=2, size=5)

As can be seen in the two plots above, the last spectral value appears to be erroneous and thus should be removed from the dataset.

## Removing data
To remove data in an index based manner the `isel` method together with `slice` can be used on the `Dataset` to select only the wanted data and the resulting `Dataset` can be assigned to a new variable.
The keywords in `isel` are the names of the axes that the selection should be applied on and `slice` is the index based selection interval.

### `slice` usage examples
```python
slice(-i)
```
will select all but the last `i` values
```python
slice(i)
```
will select all values up to `i`
```python
slice(i, None)
```
will select all values from index `i` until the end
```python
slice(i, j)
```
will select all values from index `i` until index `j`

In [ ]:
dataset_cleaned = dataset.isel(time=slice(-1))  # drop the last spectral values
dataset_cleaned.data.plot(x="time", y="spectral")

In [ ]:
last_traces = dataset_cleaned.data.isel(
    time=slice(-3, None),
)
last_traces.plot.line(x="spectral", aspect=2, size=5)

## None linear time axis
To understand why the heatmap of the data appears to be unevenly spaced we can have a look at the time axis's values over their index.

In [ ]:
import numpy as np

plt.plot(np.arange(len(dataset.time)), dataset.time)
plt.xlabel("times index")
plt.ylabel("times value (ns)")

The exponential values of the times long their index explains the "mystery" of the unevenly spaced heatmap.

### Load model and parameters

In [ ]:
PCP_model = load_model("models/PCP-model.yml")
PCP_parameters = load_parameters("models/PCP-parameters.yml")
print_md(PCP_model.validate(parameters=PCP_parameters))

In [ ]:
print_md(PCP_model)

### Create scheme and optimize it

In [ ]:
PCP_scheme = Scheme(PCP_model, PCP_parameters, {"dataset1": dataset_cleaned})
PCP_result = optimize(PCP_scheme)

In [ ]:
PCP_result.data["dataset1"]

### Result plots

In [ ]:
PCP_result.data["dataset1"]

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun

In [ ]:
!pip freeze

In [ ]:
fig = plot_overview(PCP_result.data["dataset1"], linlog=False)

As can be seen in the above plot (compared to the original one), cleaning the dataset only influenced the residual and SVD's of the residual and data, but not the SAS and DAS.

In [ ]:
print_md(PCP_result.optimized_parameters)

## chl prime

### Load model and parameters
Compared to the full data with the erroneous values for `time==78.7` the scaling of the flow to chl prime needed to be changed
from
```yaml
scaling:
  - [0.75, {vary: false}]
  - [0.25, {vary: false}]
```
to
```yaml
scaling:
  - [0.80, {vary: false}]
  - [0.20, {vary: false}]
```
in order to counter the bleaching.

In [ ]:
PCP_chl_prime_model = load_model("models/PCP-chl-prime-model.yml")
PCP_chl_prime_parameters = load_parameters("models/PCP-chl-prime-parameters-sliced-data.yml")
print_md(PCP_model.validate(parameters=PCP_chl_prime_parameters))

In [ ]:
print_md(PCP_chl_prime_model)

### Create scheme and optimize it

In [ ]:
PCP_chl_prime_scheme = Scheme(
    PCP_chl_prime_model, PCP_chl_prime_parameters, {"dataset1": dataset_cleaned}
)
PCP_chl_prime_result = optimize(PCP_chl_prime_scheme)

In [ ]:
PCP_chl_prime_result.data["dataset1"]

### Result plots

In [ ]:
%lprun fig = plot_overview(PCP_chl_prime_result.data["dataset1"], linlog=False)

In [ ]:
print_md(PCP_chl_prime_result.optimized_parameters)